In [1]:
import cv2
import mediapipe as mp
import csv
import time  # 시간 기록을 위해 추가

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 상태 변수
is_tracking = False  # 손 추적 활성화 여부

def start_tracking(event, x, y, flags, param):
    global is_tracking
    if event == cv2.EVENT_LBUTTONDOWN:  # 마우스 왼쪽 버튼 클릭
        if 50 <= x <= 200 and 50 <= y <= 100:  # 버튼 영역
            is_tracking = not is_tracking  # 상태 변경
            print("손 추적 시작" if is_tracking else "손 추적 중지")

# CSV 파일 생성
with open("hand_landmarks_with_time.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Time", "Frame", "Hand", "Landmark", "X", "Y", "Z"])  # 시간 추가

    # 웹캠 열기
    cap = cv2.VideoCapture(0)
    cv2.namedWindow("Hand Tracking")
    cv2.setMouseCallback("Hand Tracking", start_tracking)

    frame = 0  # 프레임 번호
    start_time = time.time()  # 시작 시간 기록

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("카메라 입력을 읽을 수 없습니다.")
            break

        # 버튼 그리기
        image = cv2.flip(image, 1)  # 화면 좌우 반전
        cv2.rectangle(image, (50, 50), (200, 100), (0, 255, 0), -1)  # 버튼 배경
        cv2.putText(image, "Start" if not is_tracking else "Stop", (60, 85),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # 손 추적 및 CSV 기록
        if is_tracking:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)

            # 현재 시간 계산
            current_time = time.time() - start_time

            # 랜드마크 그리기 및 기록
            if results.multi_hand_landmarks:
                for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    # 손의 라벨 (Left 또는 Right) 가져오기
                    hand_label = "Unknown"
                    if results.multi_handedness:
                        hand_label = results.multi_handedness[hand_idx].classification[0].label

                    # 랜드마크 데이터 기록 및 그리기
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    for idx, landmark in enumerate(hand_landmarks.landmark):
                        writer.writerow([current_time, frame, hand_label, idx, landmark.x, landmark.y, landmark.z])
            frame += 1

        # 화면 표시
        cv2.imshow("Hand Tracking", image)

        # 'q'를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


손 추적 시작
손 추적 중지
